In [ ]:
#importing packages to deal with pdf and docs extensions.
!pip install pdfminer.six
!pip install doc2text

     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 4.3 MB 47.1 MB/s 
  Created wheel for doc2text: filename=doc2text-0.2.4-py2.py3-none-any.whl size=8424 sha256=5cfb273fe65d7974f1015ae9d06bc335b108c140d56e388249f64ddc756f808b
  Stored in directory: /root/.cache/pip/wheels/93/81/95/e2120a7dd5316154d15614457d1b1cca0dc6c0eec232ce23a4
  Created wheel for mime: filename=mime-0.1.0-py3-none-any.whl size=39042 sha256=e043974a65d5ed7c29cf163891c79d5acdff30ce6ef0a6936ae159cc7d6787c2
  Stored in directory: /root/.cache/pip/wheels/1f/08/ca/e4b63a36aa89eb1c7ebb73976f42e9adf81dca5e02d4557b0f
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61102 sha256=1d6e109ab4c2f9ead10e5803635258509c774e00ced2555fa192685153b10d83
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built doc2text mime PyPDF2
  Attempting uninstall: Pillow
    Found existing installation: Pillow 

In [ ]:
#these functions for reading pdf and docs files and return text variable


from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()




In [ ]:
#installing spacy and languages.
!pip install spacy
!python -m spacy download en_core_web_sm


     |████████████████████████████████| 12.0 MB 19.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

#extracting names from the text
def getName(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', None, pattern)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

In [ ]:
import re

def getMobile(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [ ]:
#extracting email from the text.
import re

def getEmail(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [ ]:
#extracting skills from the text
import pandas as pd
import spacy

nlp = spacy.load('en_core_web_sm')


def getSkills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading cs that contains skills 
    data = pd.read_csv("skills.csv") 
    noun_chunks = nlp_text.noun_chunks
    # extract extract the skills
    skills = list(data.columns.values)
    
    skillset = []
    
    # getting one-gram skills (example: java)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # getting bi-grams and tri-grams (example: computer vision)
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

In [ ]:
!pip install nltk
!python -m nltk nltk.download('words')

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `python -m nltk nltk.download('words')'


In [ ]:
# extracting education
import re
import spacy
from nltk.corpus import stopwords
import nltk 
import re

# load pre-trained model
nlp = spacy.load('en_core_web_sm')
nltk.download("words")
nltk.download("stopwords")
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S',
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]

def getEducation(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]


    
   

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)

    word = 'Bachelor'
    regexp = re.compile(r'(?=.*\Bachelor\b)')
    if regexp.search(resume_text):
      education.append(word)


    return education

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Testing Cases

In [ ]:
cv1 = ""
# getting the data from the CV
for page in extract_text_from_pdf("/content/cv1.pdf"):
    cv1 += ' ' + page

In [ ]:
cv2 = ""
# getting the data from the CV
for page in extract_text_from_pdf("/content/cv2.pdf"):
    cv2 += ' ' + page

In [ ]:
#forming data representing the person
data1 = {"name" : getName(cv1), 
        "email" : getEmail(cv1),
        "number" : getMobile(cv1),
        "education" : getEducation(cv1),
        "skills" : getSkills(cv1)}

In [ ]:
#forming data representing the person
data2 = {"name" : getName(cv2), 
        "email" : getEmail(cv2),
        "number" : getMobile(cv2),
        "education" : getEducation(cv2),
        "skills" : getSkills(cv2)}

In [ ]:
#Job Requirements.
skillsRequired = ['Python', 'Pandas', "Keras", "Mathematics", "Nltk", "Machine learning", "Sql"]
education = [ 'BS', 'B.S', 'MS', 'M.S', 'Bachelor']

In [ ]:
"""
The criteria for passing the screen can be specified
the criteria here is to have ratio of the skills upto 70%
"""

def isGood(dic):
  for edu in dic['education']:
    if edu in education:

      acquired = set.intersection(set(skillsRequired), set(dic['skills']))
      ratio = len(acquired) / len(skillsRequired)
      print("the ratio of skills acquired : ", ratio)
      if ratio >0.7:
        return True

    
  return False

In [ ]:
if isGood(data1):
  print(f"The candidate {getName(cv1)} passed")
else:
  print(f"The candidate {getName(cv1)} failed")

the ratio of skills acquired :  0.2857142857142857
The candidate Mahmoud Salama failed


In [ ]:
if isGood(data2):
  print(f"The candidate {getName(cv2)} passed")
else:
  print(f"The candidate {getName(cv2)} failed")

the ratio of skills acquired :  0.7142857142857143
The candidate Muhammed Mamdouh passed


In [ ]:
#sample of the Data extracted

data1

{'education': ['Bachelor'],
 'email': 'mahmoudsalama748@yahoo.com',
 'name': 'Mahmoud Salama',
 'number': '+01276020003',
 'skills': ['English',
  'Spark',
  'System',
  'Sql',
  'Github',
  'Workflow',
  'C++',
  'Java',
  'Segmentation',
  'Research',
  'Communication',
  'Ai',
  'Cloud',
  'Coding',
  'Gis',
  'Technical',
  'Training',
  'Tensorflow',
  'Analysis',
  'Aws',
  'Email',
  'Python',
  'Engineering',
  'Shell',
  'Presentation',
  'Supervisor']}